In [6]:
# -*- coding: utf-8 -*-
"""
url: https://github.com/PlayingNumbers/ds_salary_proj
"""

import pandas as pd
import numpy as np

df_raw = pd.read_csv('glassdoor_jobs.csv')


In [16]:
df = df_raw.copy()

### salary parsing
df['hourly'] = df['Salary Estimate'].apply(lambda x: 1 if 'per hour' in x.lower() else 0)
print('hourly paid:',len(df[df['hourly']==1]))

df['who_est'] = df['Salary Estimate'].apply(lambda x: 'Employer' if '(Employer Est.)' in x else 'Glassdoor')

#df['employer_est'] = df['Salary Estimate'].apply(lambda x: 1 if '(Employer Est.)' in x else 0)
#df['glassdoor_est'] = df['Salary Estimate'].apply(lambda x: 1 if '(Glassdoor Est.)' in x else 0)
df = df[df['Salary Estimate'] != '-1']
print('No Salary:',len(df[df['Salary Estimate'] == '-1']))

salary = df['Salary Estimate'].apply(lambda x: x.split('(')[0])
# remove k and $ in salary
minus_Kd = salary.apply(lambda x: x.replace('K', '').replace('$', ''))
# remove text in salary
salary_cln = minus_Kd.apply(lambda x: x.lower().replace('per hour', '').replace('(Glassdoor Est.)', '').replace('(Employer Est.)', ''))

df['min_salary'] = salary_cln.apply(lambda x: int(x.split('-')[0]))
df['max_salary'] = salary_cln.apply(lambda x: int(x.split('-')[1]))
df['avg_salary'] = (df.min_salary + df.max_salary) / 2


hourly paid: 0
No Salary: 0
no minimum_experience_requirement: 183


In [ ]:
### company info
# Company name text only
df['company_txt'] = df.apply(lambda x: x['Company Name'] if x['Rating'] < 0 else x['Company Name'][:-3], axis=1)

# age of company
df['compnay_age'] = df.Founded.apply(lambda x: x if x < 1 else 2020 - x)



In [ ]:
### parsing of job description (python, etc.)

# python
df['Python'] = df['Job Description'].apply(lambda x: 1 if 'python' in x.lower() else 0)
# r
df['R'] = df['Job Description'].apply(lambda x: 1 if 'R' in x else 0)
# spark
df['Spark'] = df['Job Description'].apply(lambda x: 1 if 'spark' in x.lower() else 0)
# SQL
df['SQL'] = df['Job Description'].apply(lambda x: 1 if 'sql' in x.lower() else 0)
# AWS
df['AWS'] = df['Job Description'].apply(lambda x: 1 if 'aws' in x.lower() or 'amazon' in x.lower() else 0)
# excel
df['Excel'] = df['Job Description'].apply(lambda x: 1 if 'excel' in x.lower() else 0)
# Tableau
df['Tableau'] = df['Job Description'].apply(lambda x: 1 if 'tableau' in x.lower() else 0)
# PowerBI
df['PowerBI'] = df['Job Description'].apply(lambda x: 1 if 'powerbi' in x.lower() or 'power bi' in x.lower() else 0)
# Azure
df['Azure'] = df['Job Description'].apply(lambda x: 1 if 'azure' in x.lower() else 0)
# French
df['French'] = df['Job Description'].apply(lambda x: 1 if 'french' in x.lower() or 'bilingual' in x.lower() else 0)

# experience level
df['Intern'] = df['Job Title'].apply(lambda x: 1 if 'intern' in x.lower()
                                                    or 'coop' in x.lower()
                                                    or 'co-op' in x.lower()
                                                    else 0)
df['Junior'] = df['Job Title'].apply(lambda x: 1 if 'junior' in x.lower()
                                                    or 'jr' in x.lower()
                                                    or 'entry' in x.lower()
                                                    or 'new grad' in x.lower()
                                                    else 0)
df['Senior'] = df['Job Title'].apply(lambda x: 1 if 'senior' in x.lower()
                                                    or 'sr' in x.lower()
                                                    or 'staff' in x.lower()
                                                    #or 'director' in x.lower()
                                                    or 'principle' in x.lower()
                                                    or 'lead' in x.lower()
                                                    #or 'manager' in x.lower()
                                                    #or 'president' in x.lower()
                                                    or 'principal' in x.lower() ###### naiyu add
                                                    else 0)
df['Manager'] = df['Job Title'].apply(lambda x: 1 if 'director' in x.lower()
                                                    or 'manager' in x.lower()
                                                    or 'president' in x.lower()
                                                    else 0)
def get_minimum_experience_requirement(row):
    if row['Intern'] == 1:
        return 'Intern'
    elif row['Junior'] == 1:
        return 'Junior'
    elif row['Senior'] == 1:
        return 'Senior'
    elif row['Manager'] == 1:
        return 'Manager'
    else:
        return 'NA'

df['min_experience_level'] = df.apply(get_minimum_experience_requirement, axis=1)
print('no minimum_experience_requirement:',len(df[df['min_experience_level']=='NA']))

# fill na with 'Junior'
df.loc[df['min_experience_level'] == 'NA', 'min_experience_level'] = 'Junior'



In [17]:
df.dtypes

Job Title                object
Salary Estimate          object
Job Description          object
Rating                  float64
Company Name             object
Location                 object
Headquarters              int64
Size                     object
Founded                   int64
Type of ownership        object
Industry                 object
Sector                   object
Revenue                  object
Competitors               int64
Recommend Rating         object
CEO Rating               object
Career Opportunities    float64
Comp & Benefits         float64
Culture & Values        float64
Senior Management       float64
Work/Life Balance       float64
hourly                    int64
who_est                  object
min_salary                int64
max_salary                int64
avg_salary              float64
company_txt              object
compnay_age               int64
Python                    int64
R                         int64
Spark                     int64
SQL     

In [ ]:
df.to_csv('data_processed_v2.csv', index=False)